In [4]:
!pip install pandas nltk sentencepiece einops wandb joblib scikit-learn

  Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   -- ------------------------------------- 0.8/10.7 MB 5.6 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/10.7 MB 6.1 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/10.7 MB 2.7 MB/s eta 0:00:04
   -------- ------------------------------- 2.4/10.7 MB 3.1 MB/s eta 0:00:03
   ------------- -------------------------- 3.7/10.7 MB 3.9 MB/s eta 0:00:02
   ------------------- -------------------- 5.2/10.7 MB 4.5 MB/s eta 0:00:02
   ------------------------- -------------- 6.8/10.7 MB 5.1 MB/s eta 0:00:01
   -------------------------------- ------- 8.7/10.7 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------- 10.7/10.7 MB 6.1 MB/s eta 0:00:00
Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl (41.3 MB)

   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from Korpora import Korpora
import pandas as pd
from pprint import pprint
# from konlpy.tag import Mecab
from nltk.tokenize import word_tokenize as en_tokenizer
import sentencepiece as spm
import urllib.request
import csv
import numpy as np
from einops import rearrange, reduce, repeat
from torch.cuda import amp
from tqdm import tqdm
import wandb
import time
import copy
from collections import defaultdict
from sklearn.metrics import mean_squared_error
import joblib
import gc
import os


In [6]:
def makeMask(tensor, option: str) -> torch.Tensor:
    
    if option == 'padding':
        tmp = torch.full_like(tensor, fill_value=PAD_IDX).to(device) #(bs, seq_len)
        mask = (tensor != tmp).float() #(bs, seq_len)
        mask = rearrange(mask, 'bs seq_len -> bs 1 1 seq_len')
        
    elif option == 'lookahead':
        #inQ seq_len == inK seq_len
        
        padding_mask = makeMask(tensor, 'padding')
        padding_mask = np.repeat(padding_mask, 'bs 1 1 k_len -> bs 1 new k_len', new=padding_mask.size[3])
        
        mask = torch.ones_like(padding_mask)
        mask = torch.tril(mask)
        
        mask = mask * padding_mask
        
    return mask

class PositionWiseFeedForwardNetwork(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        
        #d_model = 512
        self.d_model = d_model
        #d_ff = 2048
        self.d_ff = d_ff

        self.fc1 = nn.Linear(d_model, d_ff) #512->2048 fc 512 in
        self.fc2 = nn.Linear(d_ff, d_model) #2048->512 fc 512 out
        #This consists of two linear transformations with a ReLU activation in between.
        self.relu = nn.ReLU(inplace=False)
        #Refs Residual Dropout -> We apply dropout ... for the base model we use a rate of 0.1
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, input):
        output = input
        output = self.fc1(output)
        output2 = self.relu(output)
        output2 = self.dropout(output2)
        output3 = self.fc2(output2)
        return output3

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, num_head: int):
        
        # d_model = 512 at paper
        self.d_model = d_model
        # num_head = 8 at paper
        self.num_head = num_head
        #dk = dv = d_model // h
        self.head_dim = d_model // num_head
        self.scale = torch.sqrt(torch.FloatTensor(())).to(device)
        
        self.fcQ = nn.Linear(d_model, d_model) # Q->Linear
        self.fcK = nn.Linear(d_model, d_model) # K->Linear
        self.fcV = nn.Linear(d_model, d_model) # V->Linear
        self.fcOut = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(0.1)
        #구조는 3.2 Attention Part 참조 Scaled_Dot-Product_Attention(V->Linear, K->Linear, Q->Linear)->Concat->Linear
        
        
    def forward(self, inQ, inK, inV, mask=None):
        #Saled Dot Product Attention
        
        #INPUT
        Q = self.fcQ(inQ)
        K = self.fcK(inK)
        V = self.fcV(inV)
        
        Q = rearrange(Q, 'bs seq_len (num_head head_dim) -> bs num_head seq_len head_dim', num_head=self.num_head)
        K = rearrange(K, 'bs seq_len (num_head head_dim) -> bs num_head seq_len head_dim', num_head=self.num_head)
        V = rearrange(V, 'bs seq_len (num_head head_dim) -> bs num_head seq_len head_dim', num_head=self.num_head)
        
        # Q*K_T / sqrt(dk)
        attention_energy = torch.matmul(Q,K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        #어텐션 에네르기 계산
        #어텐션 에네르기 -> Q와 K가 얼마나 관련있는지 나타내는 raw Score
        
        #Apply Masking
        if mask is not None:
            attention_energy = torch.masked_fill(attention_energy, (mask==0), -1e9)
            
        attention_score = torch.softmax(attention_energy, dim = -1)
        
        result = torch.matmul(self.dropout(attention_score), V)
        
        #Concat
        result = rearrange(result, 'bs num_head seq_len head_dim -> bs seq_len (num_head head_dim)')
        
        #Linear
        result = self.fcOut(result)
        
        return result
        
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_head, d_ff):
        super().__init__()
        
        self.d_model = d_model
        self.num_head = num_head
        self.d_ff = d_ff
        
        self.multi_head_attention = MultiHeadAttention(d_model, num_head)
        self.ffn = PositionWiseFeedForwardNetwork(d_model, d_ff)
        self.layerNorm1 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(0.1)
        self.layerNorm2 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(0.1)
        
    def forward(self, input, mask=None):
        output = self.multi_head_attention(inQ=input, inK=input, inV=input, mask=mask)
        output = self.dropout1(output)
        output = input + output
        output = self.layerNorm1(output)
        
        output_tmp = self.ffn(output)
        output_tmp = self.dropout2(output_tmp)
        output = output + output_tmp
        output = self.layerNorm2(output)
        
        return output
    
class Encoder(nn.Module):
    def __init__(self, N, d_model, num_head, d_ff, max_len=5000):
        super().__init__()
        
        #N=엔코다 레이아 반복 수
        self.N = N
        self.d_model = d_model
        self.num_head = num_head
        self.d_ff = d_ff
        
        self.embedding = nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=d_model, padding_idx=0)
        self.pos_embedding = nn.Embedding(max_len, d_model)
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_head, d_ff) for _ in range(N)])
        
        self.dropout = nn.Dropout(0.1)

    def forward(self, input, mask=None):
        batch_size = input.shape[0]
        seq_len = input.shape[1]
        
        mask = makeMask(input, 'padding')
        
        pos = torch.arange(0, seq_len).unsqueeze(0).repeat(batch_size, 1).to(device)
        
        #임베딩
        output = self.dropout(self.embedding(input) + self.pos_embedding(pos))
        
        output = self.dropout(output)
        
        for layer in self.encoder_layers:
            output = layer(output, mask)
            
        return output

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_head, d_ff):
        super().__init__()
        
        self.d_model = d_model
        self.num_head = num_head
        self.d_ff = d_ff
        
        self.multi_head_attention1 = MultiHeadAttention(d_model, num_head)
        self.layerNorm1 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(0.1)
        
        self.multi_head_attention2 = MultiHeadAttention(d_model, num_head)
        self.layerNorm2 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(0.1)
        
        self.ffn = PositionWiseFeedForwardNetwork(d_model, d_ff)
        self.layerNorm3 = nn.LayerNorm(d_model)
        self.dropout3 = nn.Dropout(0.1)
        
    def forward(self, input, enc_output, paddingMask, lookahedMask):
        
        #1
        output = self.multi_head_attention1(input, input, input, lookahedMask)
        output = self.dropout1(output)
        output = output + input
        output = self.layerNorm1(output)
        
        output_tmp = self.multi_head_attention2(output, enc_output, enc_output, paddingMask)
        output_tmp = self.dropout2(output_tmp)
        output = output + output_tmp
        output = self.layerNorm2(output)
        
        output_tmp = self.ffn(output)
        output_tmp = self.dropout3(output_tmp)
        output = output + output_tmp
        output = self.layerNorm3(output)
        
        return output

class Decoder(nn.Module):
    def __init__(self, N, d_model, num_head, d_ff, max_len=5000):
        super().__init__()
        
        self.N = N
        self.d_model = d_model
        self.num_head = num_head
        self.d_ff = d_ff
        
        self.embedding = nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=d_model, padding_idx=0)
        self.pos_embedding = nn.Embedding(max_len, d_model)
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_head, d_ff) for _ in range(N)])
        
        self.dropout = nn.Dropout(0.1)
        
        self.finalFc = nn.Linear(d_model, VOCAB_SIZE)  # Output layer to predict next token

    def forward(self, dec_input, enc_output, paddingMask, lookaheadMask):
        batch_size = dec_input.shape[0]
        seq_len = dec_input.shape[1]
        
        pos = torch.arange(0, seq_len).unsqueeze(0).repeat(batch_size, 1)
        
        output = self.dropout(self.embedding(dec_input) + self.pos_embedding(pos))
        
        # lookaheadMask = makeMask(input, 'lookahead')
        # paddingMask = makeMask(input, 'padding')

        
        for layer in self.decoder_layers:
            output = layer(output, enc_output, paddingMask, lookaheadMask)
            
        logits = self.finalFc(output)
            
        return logits
    
class Transformer(nn.Module):
    def __init__(self, N=6, d_model=512, num_head=8, d_ff=2048):
        super().__init__()
        
        self.encoder = Encoder(N, d_model, num_head, d_ff)
        self.decoder = Decoder(N, d_model, num_head, d_ff)
        
        def forward(self, enc_input, dec_input):
            # Encoder
            enc_output = self.encoder(enc_input)
            
            # Decoder
            logits, dec_output = self.decoder(dec_input, enc_output)
            
            return logits, dec_output
    

In [7]:

# if __name__ == "__main__":
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
VOCAB_SIZE = 10000
SEQ_LEN = 60


PAD_IDX = 0
BOS_IDX = 2
SOS_IDX = 3



# ENV = 'COLAB'
ENV = 'KAGGLE'
# ENV = 'SYSTEM'

# Option for Mixed Precision
FP16 = True
# FP16 = False

N = 6
D_MODEL = 256
NUM_HEAD = 8 
D_FF = 512
BATCH_SIZE = 64
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 0

CONFIG = {
    'VOCAB_SIZE': VOCAB_SIZE,
    'SEQ_LEN': SEQ_LEN,
    'N': N,
    'D_MODEL': D_MODEL,
    'NUM_HEAD': NUM_HEAD,
    'D_FF': D_FF,
    'BATCH_SIZE': BATCH_SIZE,
    'WEIGHT_DECAY' : WEIGHT_DECAY,
    'LEARNING_RATE' : LEARNING_RATE,
}

In [ ]:
import wandb
import os
# if want to run in offline mode

# os.environ["WANDB_MODE"] = "dryrun"
# wandb.init(project="Transformer_bible", entity="jiwon7258")


os.environ["WANDB_MODE"] = "online"
wandb.init(project="transformer", entity="dnjsdlf325-yeungnam-university-org", config = CONFIG, job_type = 'train')
wandb.run.name = f"train_{VOCAB_SIZE}_{SEQ_LEN}_{N}_{D_MODEL}_{D_FF}"


dataset = wandb.Artifact(f'bible-dataset_{VOCAB_SIZE}_{SEQ_LEN}', type='dataset')

wandb: Currently logged in as: dnjsdlf325 (dnjsdlf325-yeungnam-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


CommError: Error uploading run: returned error 403: {"data":{"upsertBucket":null},"errors":[{"message":"permission denied","path":["upsertBucket"],"extensions":{"code":"PERMISSION_ERROR"}}]}

In [ ]:
dataset = wandb.run.use_artifact(f'bible-dataset_{VOCAB_SIZE}_{SEQ_LEN}:latest')

# Download the artifact's contents
artifact_dir = dataset.download()

In [ ]:

# or Train / Valid Data
src_train_path = os.path.join(artifact_dir,'src_train.pkl')
src_valid_path = os.path.join(artifact_dir,'src_valid.pkl')
trg_train_path = os.path.join(artifact_dir,'trg_train.pkl')
trg_valid_path = os.path.join(artifact_dir,'trg_valid.pkl')

src_train = joblib.load(src_train_path)
src_valid = joblib.load(src_valid_path)
trg_train = joblib.load(trg_train_path)
trg_valid = joblib.load(trg_valid_path)

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, src_data, trg_data):
        super().__init__()

        assert len(src_data) == len(trg_data)

        self.src_data = src_data
        self.trg_data = trg_data

    def __len__(self):
        return len(self.src_data)
        
    def __getitem__ (self, idx):
        src = self.src_data[idx]
        trg_input = self.trg_data[idx]
        trg_output = trg_input[1:SEQ_LEN]
        trg_output = np.pad(trg_output, (0,1), 'constant', constant_values =0)
        # (seq_len,)
        return torch.Tensor(src).long(), torch.Tensor(trg_input).long(), torch.Tensor(trg_output).long()

train_dataset = TrainDataset(src_train, trg_train)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle= True, pin_memory=True)

In [ ]:
class ValidDataset(Dataset):
    def __init__(self, src_data, trg_data):
        super().__init__()

        assert len(src_data) == len(trg_data)

        self.src_data = src_data
        self.trg_data = trg_data

    def __len__(self):
        return len(self.src_data)
        
    def __getitem__ (self, idx):
        src = self.src_data[idx]
        trg_input = self.trg_data[idx]
        trg_output = trg_input[1:SEQ_LEN]
        trg_output = np.pad(trg_output, (0,1), 'constant',constant_values= 0)

        return torch.Tensor(src).long(), torch.Tensor(trg_input).long(), torch.Tensor(trg_output).long()

valid_dataset = ValidDataset(src_valid, trg_valid)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle= False, pin_memory=True)

In [ ]:
model = Transformer(N, HIDDEN_DIM, NUM_HEAD, INNER_DIM).to(device)
ic.disable()

In [ ]:
from torchsummary import summary
test1 = torch.randint(low = 0, high = 1000, size = (SEQ_LEN,))
test2 = torch.randint(low = 0, high = 1000, size = (SEQ_LEN,))
summary(model, [(SEQ_LEN,), (SEQ_LEN,)], dtypes = [torch.int, torch.int])

In [ ]:
for param in model.named_parameters():
    if 'weight' in param[0] and 'layerNorm' not in param[0] :
        torch.nn.init.xavier_uniform_(param[1])

In [ ]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = LEARNING_RATE, weight_decay = WEIGHT_DECAY)


In [ ]:

def criterion(logits: torch.tensor, targets: torch.tensor):
    return nn.CrossEntropyLoss(ignore_index=PAD_IDX)(logits.view(-1,VOCAB_SIZE), targets.view(-1))

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    # train 모드로 변경
    model.train()

    # for the Mixed Precision
    # Pytorch 예제 : https://pytorch.org/docs/stable/notes/amp_examples.html#amp-examples
    if(FP16):
        scaler = amp.GradScaler()

    dataset_size = 0
    running_loss = 0
    running_accuracy = 0
    accuracy = 0

    bar = tqdm(enumerate(dataloader), total=len(dataloader))

    for step, (src, trg_input, trg_output) in bar:
        src = src.to(device)
        trg_input = trg_input.to(device)
        trg_output = trg_output.to(device)

        batch_size = src.shape[0]

        if(FP16):
            with amp.autocast(enabled=True):
                logits, output = model(enc_src=src, dec_src=trg_input)
                loss = criterion(logits, trg_output)

                # loss를 Scale
                # Scaled Grdients를 계산(call)하기 위해 scaled loss를 backward()
                scaler.scale(loss).backward()
                # scaler.step() first unscales the gradients of the optimizer's assigned params.
                # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
                # otherwise, optimizer.step() is skipped.
                scaler.step(optimizer)

                # Updates the scale for next iteration.
                scaler.update()

        else:
            logits, output = model(enc_src=src, dec_src=trg_input)
            loss = criterion(logits, trg_output)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()

        # logits (bs, seq_len, VOCAB_SIZE)
        # trg_output (bs, seq_len)

        # zero the parameter gradients
        optimizer.zero_grad()

        # change learning rate by Scheduler
        if scheduler is not None:
            scheduler.step()

        # loss.item()은 loss를 Python Float으로 반환
        # loss.item()은 batch data의 average loss이므로, sum of loss를 구하기 위해 batch_size를 곱해준다
        running_loss += loss.item() * batch_size
        running_accuracy = np.mean(
            output.view(-1).detach().cpu().numpy() == trg_output.view(-1).detach().cpu().numpy())

        accuracy += running_accuracy

        dataset_size += batch_size
        epoch_loss = running_loss / dataset_size

        bar.set_postfix(
            Epoch=epoch, Train_Loss=epoch_loss, LR=optimizer.param_groups[0]["lr"], accuracy=accuracy / np.float(
                step+1)
        )

        # break

    accuracy /= len(dataloader)
    # Garbage Collector
    gc.collect()

    return epoch_loss, accuracy

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0
    accuracy = 0

    bar = tqdm(enumerate(dataloader), total=len(dataloader))

    for step, (src, trg_input, trg_output) in bar:
        src = src.to(device)
        trg_input = trg_input.to(device)
        trg_output = trg_output.to(device)

        batch_size = src.shape[0]

        logits, output = model(enc_src = src, dec_src = trg_input)
        loss = criterion(logits, trg_output)

        running_loss += loss.item() * batch_size
        dataset_size += batch_size

        # 실시간으로 정보를 표시하기 위한 epoch loss
        val_loss = running_loss / dataset_size
        running_accuracy = np.mean(output.view(-1).detach().cpu().numpy() == trg_output.view(-1).detach().cpu().numpy())
        
        accuracy += running_accuracy

        bar.set_postfix(
            Epoch=epoch, Valid_Loss=val_loss, LR=optimizer.param_groups[0]["lr"], accuracy = accuracy / np.float(step + 1)
        )

        # break

    accuracy /= len(dataloader)

    gc.collect()

    return val_loss, accuracy

In [ ]:
def run_training(
    model,
    optimizer,
    scheduler,
    device,
    num_epochs,
    metric_prefix="",
    file_prefix="",
    early_stopping=True,
    early_stopping_step=10,
):
    # To automatically log graidents
    wandb.watch(model, log_freq=100)

    if torch.cuda.is_available():
        print("[INFO] Using GPU:{}\n".format(torch.cuda.get_device_name()))

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf
    history = defaultdict(list)
    early_stop_counter = 0

    # num_epochs만큼, train과 val을 실행한다
    for epoch in range(1, num_epochs + 1):
        gc.collect()

        train_epoch_loss, train_accuracy = train_one_epoch(
            model,
            optimizer,
            scheduler,
            dataloader= train_dataloader,
            device=device,
            epoch=epoch,
        )

        val_loss, val_accuracy = valid_one_epoch(
            model, valid_dataloader, device=device, epoch=epoch
        )

        history[f"{metric_prefix}Train Loss"].append(train_epoch_loss)
        history[f"{metric_prefix}Train Accuracy"].append(train_accuracy)
        history[f"{metric_prefix}Valid Loss"].append(val_loss)
        history[f"{metric_prefix}Valid Accuracy"].append(val_accuracy)


        # Log the metrics
        wandb.log(
            {
                f"{metric_prefix}Train Loss": train_epoch_loss,
                f"{metric_prefix}Valid Loss": val_loss,
                f"{metric_prefix}Train Accuracy" : train_accuracy,
                f"{metric_prefix}Valid Accuracy" : val_accuracy,
            }
        )

        print(f"Valid Loss : {val_loss}")

        # deep copy the model
        if val_loss <= best_loss:
            early_stop_counter = 0

            print(
                f"Validation Loss improved( {best_loss} ---> {val_loss}  )"
            )

            # Update Best Loss
            best_loss = val_loss
            
            # Update Best Model Weight
            # run.summary['Best RMSE'] = best_loss
            best_model_wts = copy.deepcopy(model.state_dict())

            PATH = "{}epoch{:.0f}_Loss{:.4f}.bin".format(file_prefix, epoch, best_loss)
            torch.save(model.state_dict(), PATH)
            torch.save(model.state_dict(), f"{file_prefix}best_{epoch}epoch.bin")
            # Save a model file from the current directory
            wandb.save(PATH)

            print(f"Model Saved")

        elif early_stopping:
            early_stop_counter += 1
            if early_stop_counter > early_stopping_step:
                break
        
        # break


    end = time.time()
    time_elapsed = end - start
    print(
        "Training complete in {:.0f}h {:.0f}m {:.0f}s".format(
            time_elapsed // 3600,
            (time_elapsed % 3600) // 60,
            (time_elapsed % 3600) % 60,
        )
    )
    print("Best Loss: {:.4f}".format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return model, history

In [ ]:
run_training(
    model = model,
    optimizer = optimizer,
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=100, eta_min=1e-5),
    device = device,
    num_epochs = 2000,
    metric_prefix="",
    file_prefix="",
    early_stopping=True,
    early_stopping_step=10,
)

In [ ]:
torch.save(model.state_dict(), 'final.bin')
wandb.save('final.bin')

In [ ]:
wandb.finish()